In [1]:
pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/45/6d/8c1d2570a52db6263d855c3ee3daf8f4bdf4a365cd6610772d6fce5fd904/xgboost-2.0.3-py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.macosx_12_0_x86_64.whl.metadata
  Using cached xgboost-2.0.3-py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.macosx_12_0_x86_64.whl.metadata (2.0 kB)
Using cached xgboost-2.0.3-py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.macosx_12_0_x86_64.whl (2.2 MB)
Note: you may need to restart the kernel to use updated packages.


Import Libraries

In [2]:
try:
    import joblib
    import matplotlib.pyplot as plt
    %matplotlib inline
    import numpy as np
    import pandas as pd
    import seaborn as sns
    from xgboost import XGBClassifier
    from scipy import stats
    from sklearn.cluster import KMeans
    from sklearn.decomposition import PCA
    from sklearn.datasets import make_regression, make_swiss_roll
    from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.linear_model import LinearRegression, LogisticRegression
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    from sklearn.model_selection import train_test_split
    from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
    from sklearn.preprocessing import MinMaxScaler, StandardScaler
    from sklearn.svm import SVC, SVR
    from sklearn.tree import DecisionTreeClassifier

    print('All libraries imported successfully!')
except ImportError:
    print('Some libraries failed to import.')

All libraries imported successfully!


Load the Data

In [3]:
# Load data from a CSV file
df = pd.read_csv('tedsd_puf_2019.csv')
print('tedsd_puf_2019.csv loaded')
df.head()

tedsd_puf_2019.csv loaded


,DISYR,CASEID,STFIPS,CBSA2010,EDUC,MARSTAT,SERVICES,DETCRIM,LOS,PSOURCE,...,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,DIVISION,REGION,IDU,ALCDRUG
0,2019,20191553576,2,-9,4,1,7,-9,37,1,...,0,0,0,0,0,0,9,4,0,1
1,2019,20191465214,2,-9,3,1,7,-9,35,1,...,0,0,0,0,0,0,9,4,0,3
2,2019,20191443889,2,-9,2,1,7,-9,35,1,...,0,0,0,0,0,0,9,4,0,3
3,2019,20191409377,2,-9,3,1,7,-9,37,1,...,0,0,0,0,0,0,9,4,0,3
4,2019,20191479567,2,-9,3,3,7,-9,37,1,...,0,0,0,0,0,0,9,4,0,1


#### Data Exploration

In [96]:
# Summarize the data
display(df.shape)
display(df.describe())
display(df.info())

(1722503, 76)

,DISYR,CASEID,STFIPS,CBSA2010,EDUC,MARSTAT,SERVICES,DETCRIM,LOS,PSOURCE,...,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,DIVISION,REGION,IDU,ALCDRUG
count,1722503.0,1.722503e+06,1.722503e+06,1.722503e+06,1.722503e+06,1.722503e+06,1.722503e+06,1.722503e+06,1.722503e+06,1.722503e+06,...,1.722503e+06,1.722503e+06,1.722503e+06,1.722503e+06,1.722503e+06,1.722503e+06,1.722503e+06,1.722503e+06,1.722503e+06,1.722503e+06
mean,2019.0,2.019087e+10,2.558714e+01,1.817492e+04,1.777766e+00,-5.340136e-01,5.540790e+00,-6.617934e+00,2.018519e+01,2.483211e+00,...,3.146584e-04,1.067052e-03,4.600863e-03,1.085629e-03,1.239475e-03,3.434421e-02,4.596869e+00,2.420380e+00,-4.622192e-01,1.945627e+00
std,0.0,5.065654e+05,1.427222e+01,1.728295e+04,3.731331e+00,4.470499e+00,1.938862e+00,5.031978e+00,1.433813e+01,4.037795e+00,...,1.773583e-02,3.264834e-02,6.767346e-02,3.293101e-02,3.518437e-02,1.821118e-01,2.560715e+00,1.144867e+00,2.492082e+00,8.534534e-01
min,2019.0,2.019000e+10,1.000000e+00,-9.000000e+00,-9.000000e+00,-9.000000e+00,1.000000e+00,-9.000000e+00,1.000000e+00,-9.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-9.000000e+00,0.000000e+00
25%,2019.0,2.019044e+10,1.200000e+01,-9.000000e+00,2.000000e+00,1.000000e+00,4.000000e+00,-9.000000e+00,4.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00,2.000000e+00
50%,2019.0,2.019087e+10,2.700000e+01,1.674000e+04,3.000000e+00,1.000000e+00,7.000000e+00,-9.000000e+00,2.600000e+01,2.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,3.000000e+00,0.000000e+00,2.000000e+00
75%,2019.0,2.019131e+10,3.600000e+01,3.562000e+04,3.000000e+00,2.000000e+00,7.000000e+00,-9.000000e+00,3.400000e+01,6.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,3.000000e+00,0.000000e+00,3.000000e+00
max,2019.0,2.019176e+10,7.200000e+01,4.970000e+04,5.000000e+00,4.000000e+00,8.000000e+00,8.000000e+00,3.700000e+01,7.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.000000e+00,4.000000e+00,1.000000e+00,3.000000e+00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1722503 entries, 0 to 1722502
Data columns (total 76 columns):
 #   Column                 Dtype
---  ------                 -----
 0   DISYR                  int64
 1   CASEID                 int64
 2   STFIPS                 int64
 3   CBSA2010               int64
 4   EDUC                   int64
 5   MARSTAT                int64
 6   SERVICES               int64
 7   DETCRIM                int64
 8   LOS                    int64
 9   PSOURCE                int64
 10  NOPRIOR                int64
 11  ARRESTS                int64
 12  EMPLOY                 int64
 13  METHUSE                int64
 14  PSYPROB                int64
 15  PREG                   int64
 16  GENDER                 int64
 17  VET                    int64
 18  LIVARAG                int64
 19  DAYWAIT                int64
 20  SERVICES_D             int64
 21  REASON                 int64
 22  EMPLOY_D               int64
 23  LIVARAG_D              int64
 24

None

Data Cleaning

In [97]:
# # Create a copy of the data
# df1 = df.copy()
# print('Copy of the DataFrame')
# display(df1.head())

In [98]:
# # In this dataset, -9 is used to represent missing values. Which columns have -9 as a value and how many?
# missing_values = df1.isin([-9]).sum()
# display(missing_values)

In [99]:
# # What specific columns have -9 as a value?
# missing_columns = missing_values[missing_values > 0]
# display(missing_columns)

In [100]:
# # How many columns have -9 as a value?
# display(len(missing_columns))

In [101]:
# # Visualize all columns with -9 as a value
# missing_columns.plot(kind='bar', figsize=(15, 7))
# plt.title('Columns with -9 as a value')
# plt.xlabel('Columns')
# plt.ylabel('Frequency')
# plt.show()

In [102]:
# Replace -9 with NaN
# df1 = df1.replace(-9, np.nan)
# print('Replacing -9 with NaN. A -9 represents a missing value code in the dataset.')
# display(df1.head())

In [103]:
# Find columns that contain NaN values
# columns_with_nan = df1.isna().any()

# # Count the number of columns that contain NaN values
# num_columns_with_nan = columns_with_nan.sum()

# # Print the number of columns that contain NaN values
# print('There are',num_columns_with_nan,'columns that contain NaN.')

In [104]:
# Find columns that contain NaN values
# columns_with_nan = df1.columns[df1.isna().any()].tolist()

# # Calculate the percentage of NaN values in each of these columns
# nan_percentage = df1[columns_with_nan].isna().mean() * 100

# # Sort the percentages in ascending order
# nan_percentage_sorted = nan_percentage.sort_values(ascending=False)

# # Print the percentage of NaN values
# print('Percentage of NaN values in each column.')
# display(nan_percentage_sorted)

In [105]:
# Create a bar plot
# plt.figure(figsize=(20,18))
# sns.barplot(x=nan_percentage_sorted.values, y=nan_percentage_sorted.index)
# plt.title('Percentage of NaN values in columns')
# # Show the plot
# print('Bar plot showing the percentage of NaN values in columns.')
# plt.show()

In [106]:
# reason = df1['REASON']
# df1_corr = df1.corr()
# df1_corr.unstack().sort_values()
# variable = df1_corr['REASON'].sort_values()
# print('Correlation of REASON with other variables.')
# variable.tail()

In [107]:
# Create a list of colors based on the values in 'variable'
# colors = ['red' if x < 0 else 'blue' for x in variable[:-1]]

# # Visualize correlation to the target variable 'REASON'
# plt.figure(figsize=(20,18))
# variable[:-1].plot(kind='bar', color=colors)
# plt.title('Correlation to the target variable')
# print('Visualizing the positive and negative correlations of REASON with other variables.')
# plt.show()

In [108]:
# Filter to include only positive correlations
# positive_correlations = variable[variable > 0]

# # Visualize positive correlations
# plt.figure(figsize=(20,18))
# positive_correlations[:-1].plot(kind='bar', color='blue')
# plt.title('This is a visualization of only the positive correlations to the target variable')
# print('Positive correlations of REASON with other variables.')
# print("\nSERVICES is the highest positive correlation and EMPLOY is the lowest positive correlation at the moment.")
# print("\nMore exploration is needed to understand the relationship between the target variable and the features.")
# plt.show()

In [109]:
# print('These are the columns with NaN values, with the highest percentage of NaN values at the top.')
# print("\nFor example, the column FREQ3_D has roughly 83.66 precent of its values as NaN.")
# print('\nThis means that 83.66 percent of the values in the column FREQ3_D are missing or not reported.')
# display(nan_percentage_sorted)
# display(df1_corr)

In [110]:
# Define the column names from the first set
# first_set = [
#     "FREQ3_D", "DETCRIM", "FREQ3", "FRSTUSE3", "ROUTE3", "DETNLF_D", "DETNLF",
#     "FREQ2_D", "PREG", "PRIMPAY", "DAYWAIT", "HLTHINS", "FRSTUSE2", "FREQ2",
#     "ROUTE2", "CBSA2010", "PRIMINC", "FREQ1_D", "DSMCRIT", "FREQ_ATND_SELF_HELP_D",
#     "MARSTAT", "LIVARAG_D", "ARRESTS_D", "FREQ_ATND_SELF_HELP", "EMPLOY_D", "PSYPROB",
#     "SUB1_D", "FREQ1", "SUB3", "ARRESTS", "METHUSE", "EDUC", "LIVARAG", "VET", "EMPLOY",
#     "FRSTUSE1", "ROUTE1", "NOPRIOR", "IDU", "SUB2_D", "PSOURCE", "SUB1", "SUB3_D",
#     "SUB2", "ETHNIC", "RACE", "GENDER"
# ]

# Define the column names from the second set
# second_set = [
#     "DISYR", "CASEID", "STFIPS", "CBSA2010", "EDUC", "MARSTAT", "SERVICES",
#     "DETCRIM", "LOS", "PSOURCE", "TRNQFLG", "BARBFLG", "SEDHPFLG", "INHFLG",
#     "OTCFLG", "OTHERFLG", "DIVISION", "REGION", "IDU", "ALCDRUG"
# ]

# Find the common column names
# common_columns = list(set(first_set).intersection(second_set))

# # Add quotations around each column name
# first_set_quoted = [f'"{col}"' for col in first_set]
# second_set_quoted = [f'"{col}"' for col in second_set]
# common_columns_quoted = [f'"{col}"' for col in common_columns]

# # Print the results
# print("Looking for common columns in two sets of column names to see if they need be removed or filled.")
# print("\nColumn Names from the First Set:(nan_percentage_sorted)")
# print(", ".join(first_set_quoted))
# print("\nColumn Names from the Second Set:(df1_corr)")
# print(", ".join(second_set_quoted))
# print("\nCommon Column Names: (To consider for removal/fill)")
# print(", ".join(common_columns_quoted))

# plt.figure(figsize=(20,18))
# plt.barh(first_set, [1]*len(first_set), color='blue', label='First Set')
# plt.barh(second_set, [1]*len(second_set), color='red', label='Second Set')
# plt.barh(common_columns, [1]*len(common_columns), color='yellow', label='Common Columns')
# plt.xlabel('Count')
# plt.title('Column Names')
# plt.legend()
# plt.show()

# Find the Nan percentages in the common columns
# nan_percentages_common = df1[common_columns].isna().mean() * 100

# Sort and display the result
# nan_percentages_common_sorted = nan_percentages_common.sort_values(ascending=False)
# display(nan_percentages_common_sorted)

#### Data Cleaning

In [111]:
# # In column FRSTUSE1 replace -9 with 0? Confirm with value counts.
# df1['FRSTUSE1'] = df1['FRSTUSE1'].replace(-9, 0)
# print(df1['FRSTUSE1'].value_counts())

In [112]:
# # In column FRSTUSE2 replace -9 with 0? Confirm with value counts.
# df1['FRSTUSE2'] = df1['FRSTUSE2'].replace(-9, 0)
# print(df1['FRSTUSE2'].value_counts())

In [113]:
# # In column FRSTUSE3 replace -9 with 0? Confirm with value counts.
# df1['FRSTUSE3'] = df1['FRSTUSE3'].replace(-9, 0)
# print(df1['FRSTUSE3'].value_counts())

In [114]:
# # In column SUB2 replace -9 with 19? Confirm with value counts.
# df1['SUB2'] = df1['SUB2'].replace(-9, 19)
# print(df1['SUB2'].value_counts())

In [115]:
# # In column SUB2_D replace -9 with 19? Confirm with value counts.
# df1['SUB2_D'] = df1['SUB2_D'].replace(-9, 19)
# print(df1['SUB2_D'].value_counts())

In [116]:
# # In column SUB3 replace -9 with 19? Confirm with value counts.
# df1['SUB3'] = df1['SUB3'].replace(-9, 19)
# print(df1['SUB3'].value_counts())

In [117]:
# # In column SUB3_D replace -9 with 0? Confirm with value counts.
# df1['SUB3_D'] = df1['SUB3_D'].replace(-9, 0)
# print(df1['SUB3_D'].value_counts())

In [118]:
# # In column ROUTE3 replace -9 with 0? Confirm with value counts.
# df1['ROUTE3'] = df1['ROUTE3'].replace(-9, 0)
# print(df1['ROUTE3'].value_counts())

In [119]:
# # In column DSMCRIT replace -9 with 0? Confirm with value counts.
# df1['DSMCRIT'] = df1['DSMCRIT'].replace(-9, 0)
# print(df1['DSMCRIT'].value_counts())

In [120]:
# # In column PSYPROB replace -9 with 0? Confirm with value counts.
# df1['PSYPROB'] = df1['PSYPROB'].replace(-9, 0)
# print(df1['PSYPROB'].value_counts())

In [121]:
# # In column PRIMPAY replace -9 with 0? Confirm with value counts.
# df1['PRIMPAY'] = df1['PRIMPAY'].replace(-9, 0)
# print(df1['PRIMPAY'].value_counts())

In [122]:
# # In column FREQ_ATND_SELF_HELP replace -9 with 0? Confirm with value counts.
# df1['FREQ_ATND_SELF_HELP'] = df1['FREQ_ATND_SELF_HELP'].replace(-9, 0)
# print(df1['FREQ_ATND_SELF_HELP'].value_counts())

In [123]:
# # In column FREQ_ATND_SELF_HELP_D replace -9 with 0? Confirm with value counts.
# df1['FREQ_ATND_SELF_HELP_D'] = df1['FREQ_ATND_SELF_HELP_D'].replace(-9, 0)
# print(df1['FREQ_ATND_SELF_HELP_D'].value_counts())

#### Columns that need clarification:
* ROUTE2: Route of administration (secondary)
* IDU: Current IV drug use reported at admission
* HLTHINS: Health insurance


These are the 'Drug use reported at admission' columns. Not reported = 0, and reported = 1. The percentage for each column is fairly high. I believe that these columns are important for the model.

* ALCFLG: Alcohol reported at admission
* COKEFLG: Cocaine/crack reported at admission
* MARFLG: Marijuana/hashish reported at admission
* HERFLG: Heroin reported at admission
* METHFLG: Non-rx methadone reported at admission
* OPSYNFLG: Other opiates/synthetics reported at admission
* PCPFLG: PCP reported at admission
* HALLFLG: Hallucinogens reported at admission
* MTHAMFLG: Methamphetamine/speed reported at admission
* AMPHFLG: Other amphetamines reported at admission
* STIMFLG: Other stimulants reported at admission
* BENZFLG: Benzodiazepines reported at admission
* TRNQFLG: Other tranquilizers reported at admission
* BARBFLG: Barbiturates reported at admission
* SEDHPFLG: Other sedatives/hypnotics reported at admission
* INHFLG: Inhalants reported at admission
* OTCFLG: Over-the-counter medication reported at admission
* OTHERFLG: Other drug reported at admission

#### Data Preprocessing

In [124]:
# # What are the correlations of all columns to the target variable 'REASON'?
# df1_corr = df1.corr()
# variable = df1_corr['REASON'].sort_values()
# print('Correlation of REASON with other variables.')
# display(variable)

In [125]:
# # Visualize only the positive correlations
# positive_correlations = variable[variable > 0]
# plt.figure(figsize=(20,18))
# positive_correlations.plot(kind='bar', color='blue')
# plt.title('Visualization of only the positive correlations to the target variable')
# plt.show()

In [126]:
# # Display LOS
# los = df1['LOS']
# display(los)

In [127]:
# # In LOS (Length of Stay) column, replace the values with the following:
# # if days is between 1 and 30, return 1
# # if days is between 31 and 45, return 2
# # if days is between 46 and 60, return 3
# # if days is between 61 and 90, return 4
# # if days is between 91 and 120, return 5
# # if days is between 121 and 180, return 6
# # if days is between 181 and 365, return 7
# # if days is greater than 365, return 8
# # if days is any other value, return None



# def los_to_category(days):
#     if 1 <= days <= 30: 
#         return 1    
#     elif 31 <= days <= 45: 
#         return 2    
#     elif 46 <= days <= 60: 
#         return 3    
#     elif 61 <= days <= 90: 
#         return 4    
#     elif 91 <= days <= 120: 
#         return 5    
#     elif 121 <= days <= 180: 
#         return 6    
#     elif 181 <= days <= 365: 
#         return 7    
#     elif days > 365: 
#         return 8  
#     else:   
#         return None  

# # Test cases
# print(los_to_category(15))  # Expected output: 1
# print(los_to_category(35))  # Expected output: 2
# print(los_to_category(50))  # Expected output: 3
# print(los_to_category(70))  # Expected output: 4
# print(los_to_category(100))  # Expected output: 5
# print(los_to_category(150))  # Expected output: 6
# print(los_to_category(200))  # Expected output: 7
# print(los_to_category(400))  # Expected output: 8
# print(los_to_category(-5))  # Expected output: None

In [128]:
# # Check for missing values in the LOS column
# missing_values_los = los.isna().sum()
# print('Missing values in the LOS column:', missing_values_los)

In [129]:
# # Check for -9 in the LOS column
# missing_values_los = los.isin([-9]).sum()
# print('Missing values in the LOS column:', missing_values_los)

It got weird. Now we are dropping columns that are not important for the model.

In [130]:
df1 = df.copy()
# from conversation with "house of Hope recovery" class 4 which is "transfer to a different facility" would also be considered a success.
for value in df1['REASON']:
    if value == 1:
        value = value
    elif value == 4:
        value = 1
    else:
        value = 0
df1['REASON'] = df1['REASON'].replace(4, 1)
df1['REASON'] = df1['REASON'].replace(2, 0)
df1['REASON'] = df1['REASON'].replace(3, 0)
df1['REASON'] = df1['REASON'].replace(5, 0)
df1['REASON'] = df1['REASON'].replace(6, 0)
df1['REASON'] = df1['REASON'].replace(7, 0)
df1['REASON'].value_counts()

REASON
1    1095432
0     627071
Name: count, dtype: int64

In [131]:
# time to clean up the data starting with SUB1_D 
# alcohol could be a success, mmj could be a success
for value in df1['SUB1_D']:
    if value == 1 or value == 14 or value == 16 or value == 18 or value == 12 or value == 13 or value == 2:
        value = 1
    else:
        value = 0
df1['SUB1_D'] = df1['SUB1_D'].replace(14, 1)
df1['SUB1_D'] = df1['SUB1_D'].replace(16, 1)
df1['SUB1_D'] = df1['SUB1_D'].replace(18, 1)
df1['SUB1_D'] = df1['SUB1_D'].replace(12, 1)
df1['SUB1_D'] = df1['SUB1_D'].replace(13, 1)
df1['SUB1_D'] = df1['SUB1_D'].replace(2, 1)
df1['SUB1_D'] = df1['SUB1_D'].replace(3, 0)
df1['SUB1_D'] = df1['SUB1_D'].replace(4, 0)
df1['SUB1_D'] = df1['SUB1_D'].replace(5, 0)
df1['SUB1_D'] = df1['SUB1_D'].replace(6, 0)
df1['SUB1_D'] = df1['SUB1_D'].replace(7, 0)
df1['SUB1_D'] = df1['SUB1_D'].replace(8, 0)
df1['SUB1_D'] = df1['SUB1_D'].replace(9, 0)
df1['SUB1_D'] = df1['SUB1_D'].replace(10, 0)
df1['SUB1_D'] = df1['SUB1_D'].replace(11, 0)
df1['SUB1_D'] = df1['SUB1_D'].replace(15, 0)
df1['SUB1_D'] = df1['SUB1_D'].replace(17, 0)
df1['SUB1_D'] = df1['SUB1_D'].replace(19, 0)

df1['SUB1'] = df1['SUB1'].replace(14, 1)
df1['SUB1'] = df1['SUB1'].replace(16, 1)
df1['SUB1'] = df1['SUB1'].replace(18, 1)
df1['SUB1'] = df1['SUB1'].replace(12, 1)
df1['SUB1'] = df1['SUB1'].replace(13, 1)
df1['SUB1'] = df1['SUB1'].replace(2, 1)
df1['SUB1'] = df1['SUB1'].replace(3, 0)
df1['SUB1'] = df1['SUB1'].replace(4, 0)
df1['SUB1'] = df1['SUB1'].replace(5, 0)
df1['SUB1'] = df1['SUB1'].replace(6, 0)
df1['SUB1'] = df1['SUB1'].replace(7, 0)
df1['SUB1'] = df1['SUB1'].replace(8, 0)
df1['SUB1'] = df1['SUB1'].replace(9, 0)
df1['SUB1'] = df1['SUB1'].replace(10, 0)
df1['SUB1'] = df1['SUB1'].replace(11, 0)
df1['SUB1'] = df1['SUB1'].replace(15, 0)
df1['SUB1'] = df1['SUB1'].replace(17, 0)
df1['SUB1'] = df1['SUB1'].replace(19, 0)

df1['SUB2_D'] = df1['SUB2_D'].replace(14, 1)
df1['SUB2_D'] = df1['SUB2_D'].replace(16, 1)
df1['SUB2_D'] = df1['SUB2_D'].replace(18, 1)
df1['SUB2_D'] = df1['SUB2_D'].replace(12, 1)
df1['SUB2_D'] = df1['SUB2_D'].replace(13, 1)
df1['SUB2_D'] = df1['SUB2_D'].replace(2, 1)
df1['SUB2_D'] = df1['SUB2_D'].replace(3, 0)
df1['SUB2_D'] = df1['SUB2_D'].replace(4, 0)
df1['SUB2_D'] = df1['SUB2_D'].replace(5, 0)
df1['SUB2_D'] = df1['SUB2_D'].replace(6, 0)
df1['SUB2_D'] = df1['SUB2_D'].replace(7, 0)
df1['SUB2_D'] = df1['SUB2_D'].replace(8, 0)
df1['SUB2_D'] = df1['SUB2_D'].replace(9, 0)
df1['SUB2_D'] = df1['SUB2_D'].replace(10, 0)
df1['SUB2_D'] = df1['SUB2_D'].replace(11, 0)
df1['SUB2_D'] = df1['SUB2_D'].replace(15, 0)
df1['SUB2_D'] = df1['SUB2_D'].replace(17, 0)
df1['SUB2_D'] = df1['SUB2_D'].replace(19, 0)

df1['SUB2'] = df1['SUB2'].replace(14, 1)
df1['SUB2'] = df1['SUB2'].replace(16, 1)
df1['SUB2'] = df1['SUB2'].replace(18, 1)
df1['SUB2'] = df1['SUB2'].replace(12, 1)
df1['SUB2'] = df1['SUB2'].replace(13, 1)
df1['SUB2'] = df1['SUB2'].replace(2, 1)
df1['SUB2'] = df1['SUB2'].replace(3, 0)
df1['SUB2'] = df1['SUB2'].replace(4, 0)
df1['SUB2'] = df1['SUB2'].replace(5, 0)
df1['SUB2'] = df1['SUB2'].replace(6, 0)
df1['SUB2'] = df1['SUB2'].replace(7, 0)
df1['SUB2'] = df1['SUB2'].replace(8, 0)
df1['SUB2'] = df1['SUB2'].replace(9, 0)
df1['SUB2'] = df1['SUB2'].replace(10, 0)
df1['SUB2'] = df1['SUB2'].replace(11, 0)
df1['SUB2'] = df1['SUB2'].replace(15, 0)
df1['SUB2'] = df1['SUB2'].replace(17, 0)
df1['SUB2'] = df1['SUB2'].replace(19, 0)

df1['SUB3_D'] = df1['SUB3_D'].replace(14, 1)
df1['SUB3_D'] = df1['SUB3_D'].replace(16, 1)
df1['SUB3_D'] = df1['SUB3_D'].replace(18, 1)
df1['SUB3_D'] = df1['SUB3_D'].replace(12, 1)
df1['SUB3_D'] = df1['SUB3_D'].replace(13, 1)
df1['SUB3_D'] = df1['SUB3_D'].replace(2, 1)
df1['SUB3_D'] = df1['SUB3_D'].replace(3, 0)
df1['SUB3_D'] = df1['SUB3_D'].replace(4, 0)
df1['SUB3_D'] = df1['SUB3_D'].replace(5, 0)
df1['SUB3_D'] = df1['SUB3_D'].replace(6, 0)
df1['SUB3_D'] = df1['SUB3_D'].replace(7, 0)
df1['SUB3_D'] = df1['SUB3_D'].replace(8, 0)
df1['SUB3_D'] = df1['SUB3_D'].replace(9, 0)
df1['SUB3_D'] = df1['SUB3_D'].replace(10, 0)
df1['SUB3_D'] = df1['SUB3_D'].replace(11, 0)
df1['SUB3_D'] = df1['SUB3_D'].replace(15, 0)
df1['SUB3_D'] = df1['SUB3_D'].replace(17, 0)
df1['SUB3_D'] = df1['SUB3_D'].replace(19, 0)

df1['SUB3'] = df1['SUB3'].replace(14, 1)
df1['SUB3'] = df1['SUB3'].replace(16, 1)
df1['SUB3'] = df1['SUB3'].replace(18, 1)
df1['SUB3'] = df1['SUB3'].replace(12, 1)
df1['SUB3'] = df1['SUB3'].replace(13, 1)
df1['SUB3'] = df1['SUB3'].replace(2, 1)
df1['SUB3'] = df1['SUB3'].replace(3, 0)
df1['SUB3'] = df1['SUB3'].replace(4, 0)
df1['SUB3'] = df1['SUB3'].replace(5, 0)
df1['SUB3'] = df1['SUB3'].replace(6, 0)
df1['SUB3'] = df1['SUB3'].replace(7, 0)
df1['SUB3'] = df1['SUB3'].replace(8, 0)
df1['SUB3'] = df1['SUB3'].replace(9, 0)
df1['SUB3'] = df1['SUB3'].replace(10, 0)
df1['SUB3'] = df1['SUB3'].replace(11, 0)
df1['SUB3'] = df1['SUB3'].replace(15, 0)
df1['SUB3'] = df1['SUB3'].replace(17, 0)
df1['SUB3'] = df1['SUB3'].replace(19, 0)
# df1['SUB1_D'].notna().value_counts()
# time to clean up the data starting with SUB1 
# alcohol could be a success, mmj could be a success, using the same values for SUB1_D
for value in df1['SUB1']:
    if value == 1 or value == 14 or value == 16 or value == 18 or value == 12 or value == 13 or value == 2:
        value = 1
    else:
        value = 0
for value in df1['SUB2_D']:
    if value == 1 or value == 14 or value == 16 or value == 18 or value == 12 or value == 13 or value == 2:
        value = 1
    else:
        value = 0
# df1['SUB1_D'].notna().value_counts()
# time to clean up the data starting with SUB1 
# alcohol could be a success, mmj could be a success, using the same values for SUB1_D
for value in df1['SUB2']:
    if value == 1 or value == 14 or value == 16 or value == 18 or value == 12 or value == 13 or value == 2:
        value = 1
    else:
        value = 0
for value in df1['SUB3_D']:
    if value == 1 or value == 14 or value == 16 or value == 18 or value == 12 or value == 13 or value == 2:
        value = 1
    else:
        value = 0
# df1['SUB1_D'].notna().value_counts()
# time to clean up the data starting with SUB1 
# alcohol could be a success, mmj could be a success, using the same values for SUB1_D
for value in df1['SUB3']:
    if value == 1 or value == 14 or value == 16 or value == 18 or value == 12 or value == 13 or value == 2:
        value = 1
    else:
        value = 0
df1['SUB1'] = df1['SUB1'].replace(-9, 0)
df1['SUB1_D'] = df1['SUB1_D'].replace(-9, 0)
df1['SUB2'] = df1['SUB2'].replace(-9, 0)
df1['SUB2_D'] = df1['SUB2_D'].replace(-9, 0)
df1['SUB3'] = df1['SUB3'].replace(-9, 0)
df1['SUB3_D'] = df1['SUB3_D'].replace(-9, 0)
print(df1['SUB1'].value_counts())
print(df1['SUB1_D'].value_counts())
print(df1['SUB2'].value_counts())
print(df1['SUB2_D'].value_counts())
print(df1['SUB3'].value_counts())
print(df1['SUB3_D'].value_counts())

# df1['SUB1_D'].notna().value_counts()
# # In column SUB2 replace -9 with 19
df1['SUB2'] = df1['SUB2'].replace(-9, 19)
# print(df1['SUB2'].value_counts())
# # In column SUB2_D replace -9 with 19
# # df1['SUB2_D'] = df1['SUB2_D'].replace(-9, 19)
# print(df1['SUB2_D'].value_counts())
# # In column SUB3 replace -9 with 19
# # df1['SUB3'] = df1['SUB3'].replace(-9, 19)
# print(df1['SUB3'].value_counts())
# # In column SUB3_D replace -9 with 0
# # df1['SUB3_D'] = df1['SUB3_D'].replace(-9, 0)
# print(df1['SUB3_D'].value_counts())

SUB1
0    1145410
1     577093
Name: count, dtype: int64
SUB1_D
0    1153301
1     569202
Name: count, dtype: int64
SUB2
1    959459
0    763044
Name: count, dtype: int64
SUB2_D
1    990170
0    732333
Name: count, dtype: int64
SUB3
1    1278583
0     443920
Name: count, dtype: int64
SUB3_D
1    1403980
0     318523
Name: count, dtype: int64


In [132]:
# time to clean up the data with FREQ1_D, FREQ2_D, FREQ3_D
# replacing -9 in the columns with 1 since 1 indicates 'no use' and if data was missing we assumed it was irrelivent to begin with
df1['FREQ1_D'] = df1['FREQ1_D'].replace(-9, 1)
df1['FREQ2_D'] = df1['FREQ2_D'].replace(-9, 1)
df1['FREQ3_D'] = df1['FREQ3_D'].replace(-9, 1)

In [133]:
# time to clean up the data with FREQ1, FREQ2, FREQ3
# replacing -9 in the columns with 1 since 1 indicates 'no use' and if data was missing we assumed it was irrelivent to begin with
df1['FREQ1'] = df1['FREQ1'].replace(-9, 1)
df1['FREQ2'] = df1['FREQ2'].replace(-9, 1)
df1['FREQ3'] = df1['FREQ3'].replace(-9, 1)

In [134]:
# time to clean up the data with 'CASEID'
df1['CASEID'].notna().value_counts()
# CASEID has no null values and does need info filled
# we can drop this field
df1 = df1.drop(columns="CASEID")

In [135]:
# time to clean up the data with 'AGE'
df1['AGE'].notna().value_counts()
# AGE has no null values and does need info filled

AGE
True    1722503
Name: count, dtype: int64

In [136]:
# time to clean up the data with 'GENDER'
# gender is not a columns where we can fill in the data and not decrease the accuracy. so for all -9 values I am replacing with 0
df1['GENDER'] = df1['GENDER'].replace(-9, 0)
df1['GENDER'].notna().value_counts()

GENDER
True    1722503
Name: count, dtype: int64

In [137]:
# time to clean up the data with 'DISYR'
df1['DISYR'].notna().value_counts()
# DISYR has no null values and does need info filled
# drop this column
df1 = df1.drop(columns="DISYR")

In [138]:
# time to clean up the data with 'RACE'
# for race, '7' indicateds 'other single race  so for all -9 values i replaces them with 7
df1['RACE'] = df1['RACE'].replace(-9, 7)
df1['RACE'].notna().value_counts()

RACE
True    1722503
Name: count, dtype: int64

In [139]:
# time to clean up the data with 'ETHNIC'
# for ETHNIC, '4' indicateds 'other single race  so for all -9 values I replaced them with 4
df1['ETHNIC'] = df1['ETHNIC'].replace(-9, 4)
df1['ETHNIC'].notna().value_counts()

ETHNIC
True    1722503
Name: count, dtype: int64

In [140]:
# time to clean up the data with 'MARSTAT'
# for MARSTAT, hard to determine what to fill -9 with so filling with 0 for now
df1['MARSTAT'] = df1['MARSTAT'].replace(-9, 0)
df1['MARSTAT'].notna().value_counts()

MARSTAT
True    1722503
Name: count, dtype: int64

In [141]:
# time to clean up the data with 'EDUC'
# for EDUC, hard to determine what to fill -9 with. options are 1-5 so filling with 2 as it seems like a fair average
df1['EDUC'] = df1['EDUC'].replace(-9, 2)
df1['EDUC'].notna().value_counts()

EDUC
True    1722503
Name: count, dtype: int64

In [142]:
# time to clean up the data with 'EMPLOY' and 'EMPLOY_D'
# for EMPLOY, hard to determine what to fill -9 with. options are 1-4. filling with 2 for now since if the data is missing is seems like that would indicate 'unemployed'
df1['EMPLOY'] = df1['EMPLOY'].replace(-9, 0)
df1['EMPLOY_D'] = df1['EMPLOY_D'].replace(-9, 0)
df1['EMPLOY'].notna().value_counts()

EMPLOY
True    1722503
Name: count, dtype: int64

In [143]:
# time to clean up the data with 'DETNLF' and 'DETNLF_D'
# for DETNLF, its just details on the previous columns not in labor force option so im replacing missing values with 0 as to not affect the results as much. we may want to drop this column
# df1['DETNLF'] = df1['DETNLF'].replace(-9, 0)
# df1['DETNLF_D'] = df1['DETNLF_D'].replace(-9, 0)
# df1['DETNLF'].notna().value_counts()
# drop this column
df1 = df1.drop(columns="DETNLF")
df1 = df1.drop(columns="DETNLF_D")

In [144]:
# time to clean up the data with 'PREG'
# for PREG, -9 most likely indicates male patients. replacing with 0 as to not affect the results as much. 
df1['PREG'] = df1['PREG'].replace(-9, 2)
df1['PREG'].notna().value_counts()

PREG
True    1722503
Name: count, dtype: int64

In [145]:
# time to clean up the data with 'VET'
# for VET, -9 most likely indicates not a veteren so im replacing -9 with 2 for 'no'
df1['VET'] = df1['VET'].replace(-9, 2)
df1['VET'].value_counts()

VET
2    1679845
1      42658
Name: count, dtype: int64

In [146]:
# time to clean up the data with 'LIVARAG'
# for LIVARAG, its hard to determine what -9 could indicate so im replacing -9 with 0 as to not affect the data as much with the -9's in this column
df1['LIVARAG'] = df1['LIVARAG'].replace(-9, 1)
df1['LIVARAG_D'] = df1['LIVARAG_D'].replace(-9, 1)
df1['LIVARAG'].value_counts()

LIVARAG
3    1013787
1     436264
2     272452
Name: count, dtype: int64

In [147]:
# time to clean up the data with 'PRIMINC'
# for PRIMINC, its hard to determine what -9 could indicate so im replacing -9 with 0 as to not affect the data as much with the -9's in this column
df1['PRIMINC'] = df1['PRIMINC'].replace(-9, 4)
df1['PRIMINC'].value_counts()

PRIMINC
4    855478
5    397327
1    312986
2     81605
3     75107
Name: count, dtype: int64

In [148]:
# time to clean up the data with 'ARRESTS' and 'ARRESTS_D'
# for 'ARRESTS' and 'ARRESTS_D', 0 indicates none so im setting -9 to none as that seems most likely to be the case if the information is missing
df1['ARRESTS'] = df1['ARRESTS'].replace(-9, 0)
df1['ARRESTS_D'] = df1['ARRESTS_D'].replace(-9, 0)
df1['ARRESTS'].value_counts()
df1['ARRESTS_D'].value_counts()

ARRESTS_D
0    1643399
1      61908
2      17196
Name: count, dtype: int64

In [149]:
# time to clean up the data with ' STFIPS'
# for ' STFIPS', there are no null values in this column but we may want to remove it because the large values could through the accuracy off
df1['STFIPS'].notna().value_counts()
# drop this column
df1 = df1.drop(columns="STFIPS")

In [150]:
# time to clean up the data with 'REGION'
# for 'REGION', there are no null values in this column but we may want to remove it because the values could through the accuracy off and it doesnt seem to be relevant data
df1['REGION'].value_counts()

REGION
1    532413
3    505555
4    378260
2    303497
0      2778
Name: count, dtype: int64

In [151]:
# time to clean up the data with 'DIVISION'
# for 'DIVISION', there are no null values in this column but we may want to remove it because the values could through the accuracy off and it doesnt seem to be relevant data
df1['DIVISION'].value_counts()

DIVISION
2    372934
5    331617
8    229376
1    159479
4    152327
3    151170
9    148884
6    100660
7     73278
0      2778
Name: count, dtype: int64

In [152]:
# time to clean up the data with 'SERVICES and SERVICES_D'
# for 'SERVICES and SERVICES_D', there are no null values in this column but we may want to remove it because the values could through the accuracy off and it doesnt seem to be relevant data for the questions we are trying to answer
df1['SERVICES'].value_counts()
df1['SERVICES_D'].value_counts()

SERVICES_D
7    884266
2    240303
6    235771
4    179737
5    121470
1     43446
8     13267
3      4243
Name: count, dtype: int64

In [153]:
# time to clean up the data with 'DAYWAIT'
# for 'DAYWAIT', its seems safe to replace a value of missing data with a value of '0' to indicate that they didnt wait
df1['DAYWAIT'] = df1['DAYWAIT'].replace(-9, 0)
df1['DAYWAIT'].value_counts()

DAYWAIT
0    1469735
1     175018
2      34722
3      26618
4      16410
Name: count, dtype: int64

In [154]:
# time to clean up the data with 'METHUSE'
# for 'METHUSE', its seems safe to replace a value of missing data with a value of 'none'
df1['METHUSE'] = df1['METHUSE'].replace(-9, 0)
df1['METHUSE'].value_counts()

METHUSE
2    1327459
1     216239
0     178805
Name: count, dtype: int64

In [155]:
# time to clean up the data with 'LOS'
# for 'LOS', stands for length of stay this data seems irrelivent to the questions we are trying to answer so I suggest dropping the column to perserve prediction accuracy
# 1-30, 31-45, 46-60, 61-90, 91-120, 121-180, 181-365, more than
def los_to_category(days):
    if days <= 30:
        return 1
    elif days <= 31:
        return 2
    elif days <= 32:
        return 3
    elif days <= 33:
        return 4
    elif days <= 34:
        return 5
    elif days <= 35:
        return 6
    elif days <= 36:
        return 7
    else:
        return 8
# Test cases

# df1['LOS'].value_counts()

In [156]:
print(los_to_category(15))  # Expected output: 1
print(los_to_category(35))  # Expected output: 2
print(los_to_category(50))  # Expected output: 3
print(los_to_category(70))  # Expected output: 4
print(los_to_category(100))  # Expected output: 5
print(los_to_category(150))  # Expected output: 6
print(los_to_category(200))  # Expected output: 7
print(los_to_category(400))  # Expected output: 8
print(los_to_category(-5))  # Expected output: None

1
6
8
8
8
8
8
8
1


In [157]:
df1['LOS'] = df1['LOS'].apply(los_to_category)

In [158]:
df1['LOS'].value_counts()

LOS
1    934659
4    140555
7    128974
5    114387
6    114266
2    111830
8     93500
3     84332
Name: count, dtype: int64

In [159]:
# time to clean up the data with PSOURCE: Referral source
# for 'PSOURCE' this doesnt seem to me to be relevant data so im replacing -9 with 1 for now to indicate that it was self motivated
df1['PSOURCE'] = df1['PSOURCE'].replace(-9, 1)

In [160]:
# time to clean up the data with DETCRIM: Detailed criminal justice referral
# for 'DETCRIM' im replacing -9 with 0 for now. most of this data is missing. we may want to drop this column
df1['DETCRIM'] = df1['DETCRIM'].replace(-9, 0)
# drop this column
df1 = df1.drop(columns="DETCRIM")

In [161]:
# time to clean up the data with NOPRIOR: Previous substance use treatment episodes
# for 'NOPRIOR' im replacing -9 with 0 for now. its could go either way. we will ask clients
df1['NOPRIOR'] = df1['NOPRIOR'].replace(-9, 0)

In [162]:
# time to clean up the data with ROUTE1: Route of administration (primary)
# time to clean up the data with ROUTE2: Route of administration (secondary)
# time to clean up the data with ROUTE3: Route of administration (tertiary)
# for 'ROUTE1' im replacing -9 with 5 for now to indicate 'other'. its could go either way. we will ask clients
df1['ROUTE1'] = df1['ROUTE1'].replace(-9, 5)
df1['ROUTE2'] = df1['ROUTE2'].replace(-9, 5)
df1['ROUTE3'] = df1['ROUTE3'].replace(-9, 5)

In [163]:
# time to clean up the data with FRSTUSE1: Age at first use (primary)
# FRSTUSE1 is a column where its hard to determine what a value of -9 should be replaced with so im replacing it with 0 for now to have less of an impact on the data
df1['FRSTUSE1'] = df1['FRSTUSE1'].replace(-9, 3)
# adding Beau's code
# In column FRSTUSE2 replace -9 with 0
df1['FRSTUSE2'] = df1['FRSTUSE2'].replace(-9, 3)
print(df1['FRSTUSE2'].value_counts())
# In column FRSTUSE3 replace -9 with 0
df1['FRSTUSE3'] = df1['FRSTUSE3'].replace(-9, 3)
print(df1['FRSTUSE3'].value_counts())

FRSTUSE2
3    1067078
2     170892
4     153705
7     103384
5      95617
6      78339
1      53488
Name: count, dtype: int64
FRSTUSE3
3    1464306
2      78630
4      57055
7      37209
5      32205
6      27655
1      25443
Name: count, dtype: int64


In [164]:
# In column SUB2 replace -9 with 19
df1['SUB2'] = df1['SUB2'].replace(-9, 19)
print(df1['SUB2'].value_counts())
# In column SUB2_D replace -9 with 19
df1['SUB2_D'] = df1['SUB2_D'].replace(-9, 19)
print(df1['SUB2_D'].value_counts())

SUB2
1    959459
0    763044
Name: count, dtype: int64
SUB2_D
1    990170
0    732333
Name: count, dtype: int64


In [165]:
# In column SUB3 replace -9 with 19
df1['SUB3'] = df1['SUB3'].replace(-9, 19)
print(df1['SUB3'].value_counts())
# In column SUB3_D replace -9 with 0
df1['SUB3_D'] = df1['SUB3_D'].replace(-9, 0)
print(df1['SUB3_D'].value_counts())

SUB3
1    1278583
0     443920
Name: count, dtype: int64
SUB3_D
1    1403980
0     318523
Name: count, dtype: int64


In [166]:
# In column ROUTE3 replace -9 with 0
df1['ROUTE3'] = df1['ROUTE3'].replace(-9, 0)
print(df1['ROUTE3'].value_counts())

ROUTE3
5    1365958
2     154215
1     122303
3      48044
4      31983
Name: count, dtype: int64


In [167]:
# In column DSMCRIT replace -9 with 0
df1['DSMCRIT'] = df1['DSMCRIT'].replace(-9, 5)
print(df1['DSMCRIT'].value_counts())

DSMCRIT
5     790823
4     289535
8     143441
19    125445
7      82105
6      56213
9      52972
2      40101
10     39565
3      29130
11     23297
12     17972
1      12628
13      7786
15      5239
17      2246
14      2181
16      1490
18       334
Name: count, dtype: int64


In [168]:
# In column PSYPROB replace -9 with 1
df1['PSYPROB'] = df1['PSYPROB'].replace(-9, 1)
print(df1['PSYPROB'].value_counts())

PSYPROB
1    899280
2    823223
Name: count, dtype: int64


In [169]:
# In column PRIMPAY replace -9 with 1
df1['PRIMPAY'] = df1['PRIMPAY'].replace(-9, 1)
print(df1['PRIMPAY'].value_counts())

PRIMPAY
1    1038444
4     389616
5     162022
7      49127
2      44071
6      20758
3      18465
Name: count, dtype: int64


In [170]:
# In column FREQ_ATND_SELF_HELP replace -9 with 0
df1['FREQ_ATND_SELF_HELP'] = df1['FREQ_ATND_SELF_HELP'].replace(-9, 3)
print(df1['FREQ_ATND_SELF_HELP'].value_counts())
# In column FREQ_ATND_SELF_HELP_D replace -9 with 0
df1['FREQ_ATND_SELF_HELP_D'] = df1['FREQ_ATND_SELF_HELP_D'].replace(-9, 3)
print(df1['FREQ_ATND_SELF_HELP_D'].value_counts())

FREQ_ATND_SELF_HELP
1    1037604
3     389036
4     126267
2     104522
5      65074
Name: count, dtype: int64
FREQ_ATND_SELF_HELP_D
1    859331
3    452977
4    186509
2    115895
5    107791
Name: count, dtype: int64


In [171]:
reason = df1['REASON']
df1_corr = df1.corr()
df1_corr.unstack().sort_values()
variable = df1_corr['REASON'].sort_values()
variable.tail(20)

ALCFLG       0.025359
LIVARAG_D    0.026111
FREQ1_D      0.029926
HLTHINS      0.032940
ROUTE3       0.036662
EMPLOY_D     0.040257
ETHNIC       0.057235
SUB1_D       0.066446
PRIMPAY      0.067499
AGE          0.068791
ROUTE2       0.071289
SUB1         0.073811
SUB2         0.080155
SUB3_D       0.088915
METHUSE      0.096050
SUB3         0.102280
DIVISION     0.128182
SUB2_D       0.136620
REGION       0.153238
REASON       1.000000
Name: REASON, dtype: float64

In [172]:
# Display df1
display(df1)

,CBSA2010,EDUC,MARSTAT,SERVICES,LOS,PSOURCE,NOPRIOR,ARRESTS,EMPLOY,METHUSE,...,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,DIVISION,REGION,IDU,ALCDRUG
0,-9,4,1,7,8,1,0,0,2,2,...,0,0,0,0,0,0,9,4,0,1
1,-9,3,1,7,6,1,0,0,1,2,...,0,0,0,0,0,0,9,4,0,3
2,-9,2,1,7,6,1,0,0,4,2,...,0,0,0,0,0,0,9,4,0,3
3,-9,3,1,7,8,1,1,0,3,2,...,0,0,0,0,0,0,9,4,0,3
4,-9,3,3,7,8,1,1,0,4,2,...,0,0,0,0,0,0,9,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722498,-9,4,2,7,4,1,0,0,1,2,...,0,0,0,0,0,0,8,4,0,1
1722499,-9,3,3,7,1,7,0,0,4,2,...,0,0,0,0,0,0,8,4,0,2
1722500,-9,2,1,6,4,1,0,0,3,2,...,0,0,0,0,0,0,8,4,0,2
1722501,-9,1,1,7,1,7,0,0,4,2,...,0,0,0,0,0,0,8,4,0,3


In [173]:
# What columns are highly correlated with the target variable 'REASON'?
high_corr_df = df1_corr['REASON'].sort_values()
print('Correlation of REASON with other variables.')
display(high_corr_df.tail(21))

Correlation of REASON with other variables.


ROUTE1       0.024558
ALCFLG       0.025359
LIVARAG_D    0.026111
FREQ1_D      0.029926
HLTHINS      0.032940
ROUTE3       0.036662
EMPLOY_D     0.040257
ETHNIC       0.057235
SUB1_D       0.066446
PRIMPAY      0.067499
AGE          0.068791
ROUTE2       0.071289
SUB1         0.073811
SUB2         0.080155
SUB3_D       0.088915
METHUSE      0.096050
SUB3         0.102280
DIVISION     0.128182
SUB2_D       0.136620
REGION       0.153238
REASON       1.000000
Name: REASON, dtype: float64

#### Random Forest Regressor Outputs:

* First Run:
    * Training Accuracy: 0.9980399157620962
    * Testing Accuracy: 0.837835013541325

* Second Run:
    * Training Accuracy: 
    * Testing Accuracy: 


In [178]:
# Drop the target variable 'REASON' from the dataset
X = df1.drop(columns='REASON')
y = df1['REASON']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit on the training data
X_train = scaler.fit_transform(X_train)

# Transform the testing data
X_test = scaler.transform(X_test)

# Create a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=2, min_samples_leaf=4, random_state=42)
et_model = ExtraTreesClassifier(n_estimators=100, max_depth=10, min_samples_split=2, min_samples_leaf=4, random_state=42)


# # Train the model
# rf_model.fit(X_train, y_train)

# # Make predictions on the training set
# y_train_pred = rf_model.predict(X_train)

# # Calculate the training accuracy
# train_accuracy = accuracy_score(y_train, y_train_pred)

# # Make predictions on the testing set
# y_test_pred = rf_model.predict(X_test)

# # Calculate the testing accuracy
# test_accuracy = accuracy_score(y_test, y_test_pred)

# # Print the accuracies
# print('Training Accuracy:', train_accuracy)
# print('Testing Accuracy:', test_accuracy)

# Train the RandomForest model
rf_model.fit(X_train, y_train)

# Make predictions with the RandomForest model
y_train_pred_rf = rf_model.predict(X_train)
y_test_pred_rf = rf_model.predict(X_test)

# Calculate the RandomForest accuracies
train_accuracy_rf = accuracy_score(y_train, y_train_pred_rf)
test_accuracy_rf = accuracy_score(y_test, y_test_pred_rf)

# Print the RandomForest accuracies
print('RandomForest Training Accuracy:', train_accuracy_rf)
print('RandomForest Testing Accuracy:', test_accuracy_rf)

# Train the ExtraTrees model
et_model.fit(X_train, y_train)

# Make predictions with the ExtraTrees model
y_train_pred_et = et_model.predict(X_train)
y_test_pred_et = et_model.predict(X_test)

# Calculate the ExtraTrees accuracies
train_accuracy_et = accuracy_score(y_train, y_train_pred_et)
test_accuracy_et = accuracy_score(y_test, y_test_pred_et)

# Print the ExtraTrees accuracies
print('ExtraTrees Training Accuracy:', train_accuracy_et)
print('ExtraTrees Testing Accuracy:', test_accuracy_et)

RandomForest Training Accuracy: 0.7882601041217647
RandomForest Testing Accuracy: 0.787161720865832
ExtraTrees Training Accuracy: 0.7665141269751423
ExtraTrees Testing Accuracy: 0.7649905225238823


In [180]:
# Print the RandomForest accuracies
print('RandomForest Training Accuracy:', train_accuracy_rf)
print('RandomForest Testing Accuracy:', test_accuracy_rf)

# Check for overfitting and underfitting in RandomForest
if train_accuracy_rf > test_accuracy_rf:
    print('RandomForest is overfitting.')
elif train_accuracy_rf < test_accuracy_rf:
    print('RandomForest is underfitting.')
else:
    print('RandomForest is fitting well.')

# Print the ExtraTrees accuracies
print('ExtraTrees Training Accuracy:', train_accuracy_et)
print('ExtraTrees Testing Accuracy:', test_accuracy_et)

# Check for overfitting and underfitting in ExtraTrees
if train_accuracy_et > test_accuracy_et:
    print('ExtraTrees is overfitting.')
elif train_accuracy_et < test_accuracy_et:
    print('ExtraTrees is underfitting.')
else:
    print('ExtraTrees is fitting well.')

RandomForest Training Accuracy: 0.7882601041217647
RandomForest Testing Accuracy: 0.787161720865832
RandomForest is overfitting.
ExtraTrees Training Accuracy: 0.7665141269751423
ExtraTrees Testing Accuracy: 0.7649905225238823
ExtraTrees is overfitting.


In [181]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation on the RandomForest model
rf_scores = cross_val_score(rf_model, X_train, y_train, cv=5)
print('RandomForest Cross-Validation Accuracy:', np.mean(rf_scores))

# Perform cross-validation on the ExtraTrees model
et_scores = cross_val_score(et_model, X_train, y_train, cv=5)
print('ExtraTrees Cross-Validation Accuracy:', np.mean(et_scores))

RandomForest Cross-Validation Accuracy: 0.7866280319297335
ExtraTrees Cross-Validation Accuracy: 0.7629836538765005


In [ ]:
# from sklearn.model_selection import GridSearchCV

# # Define the parameter grid
# param_grid = {
#     'n_estimators': [100, 200, 300, 400, 500],
#     'max_depth': [None, 10, 20, 30, 40, 50],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'bootstrap': [True, False]
# }

# # Initialize the GridSearchCV
# grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

# # Fit the GridSearchCV
# grid_search.fit(X_train, y_train)

# # Get the best parameters
# best_params = grid_search.best_params_

# # Train a RandomForestClassifier with the best parameters
# rf_best = RandomForestClassifier(**best_params, random_state=42)
# rf_best.fit(X_train, y_train)

# # Make predictions on the training set
# y_train_pred = rf_best.predict(X_train)

# # Calculate the training accuracy
# train_accuracy = accuracy_score(y_train, y_train_pred)

# # Make predictions on the testing set
# y_test_pred = rf_best.predict(X_test)

# # Calculate the testing accuracy
# test_accuracy = accuracy_score(y_test, y_test_pred)

# # Print the accuracies
# print('Training Accuracy:', train_accuracy)
# print('Testing Accuracy:', test_accuracy)

Fitting 3 folds for each of 540 candidates, totalling 1620 fits
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=18.9min
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=25.7min
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=29.4min


KeyboardInterrupt: 